<a href="https://colab.research.google.com/github/PPRFNKrec/fastdiffusion/blob/master/DRUMANDBASSANALYSIS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**УСТАНОВИТЕ ЗАВИСИМОСТИ**

In [ ]:
!pip install ipywidgets
!pip install librosa
!pip install pyloudnorm
!pip install audioread
!pip install scipy
!pip install mir_eval

In [ ]:
import io
import librosa
import librosa.effects
import numpy as np
import scipy.signal
from scipy import stats
from scipy.signal import find_peaks
from pyloudnorm import Meter
from librosa import feature, onset, display
from mir_eval import separation

def analyze_stereo(y):
    panning = np.mean(separation.bss_eval_sources(y, y, compute_permutation=False)[2])
    channel_correlation = np.corrcoef(y[0], y[1])[0][1]
    return panning, channel_correlation

def analyze_loudness_ebu(y_mono, sr, target_loudness=0):
    meter = Meter(sr)
    loudness = meter.integrated_loudness(y_mono)
    return loudness - target_loudness

def dynamic_range_analysis(y_mono, sr):
    hop_length = 512
    frame_length = 2048
    S = np.abs(librosa.stft(y_mono, n_fft=frame_length, hop_length=hop_length))
    
    # Calculate the short-term loudness
    short_term_loudness = librosa.feature.rms(S=S, frame_length=frame_length, hop_length=hop_length)
    
    # Calculate the loudness range
    lra = np.max(short_term_loudness) - np.min(short_term_loudness)
    
    return lra

def harmonic_analysis(y_mono, sr):
    chroma = librosa.feature.chroma_stft(y=y_mono, sr=sr)
    key, key_strength = stats.mode(np.argmax(chroma, axis=0), keepdims=False)
    return key, key_strength.item()

def analyze_attack_release(y_mono, sr):
    onset_env = librosa.onset.onset_strength(y=y_mono, sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=onset_env, sr=sr)
    onset_times = librosa.frames_to_time(onset_frames, sr=sr)
    attack_times = np.diff(onset_times)

    if len(attack_times) > 0:
        mean_attack_time = np.mean(attack_times)
    else:
        mean_attack_time = 0

    hop_length = 512
    o_env = librosa.onset.onset_strength(y=y_mono, sr=sr, hop_length=hop_length)
    release_frames = np.argwhere(o_env > np.max(o_env) / 10).flatten()
    release_samples = librosa.frames_to_samples(release_frames, hop_length=hop_length)
    mean_release_time = np.mean(np.diff(release_samples)) / sr

    return mean_attack_time, mean_release_time

def analyze_waveform_shape(y_mono):
    curvature = np.mean(np.gradient(np.gradient(y_mono)))
    flatness = np.mean(librosa.feature.spectral_flatness(y=y_mono))
    return curvature, flatness

def analyze_sound_scene(y_mono, sr):
    pass  # Реализация алгоритма анализа звуковой сцены может быть сложной и выходить за рамки данного ответа.

def analyze_volume_variation(y_mono):
    rms = librosa.feature.rms(y=y_mono)
    coef_variation = np.std(rms) / np.mean(rms)
    return coef_variation

def analyze_music_structure(y_mono, sr):
    # Тембровая сегментация
    S = np.abs(librosa.stft(y_mono))
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    segments_timbre = librosa.segment.agglomerative(contrast, 20)

    # Гармоническая сегментация
    chroma = librosa.feature.chroma_stft(y=y_mono, sr=sr)
    segments_harmony = librosa.segment.agglomerative(chroma, 20)

    # Сегментация по громкости
    rms = librosa.feature.rms(y=y_mono)
    segments_loudness = librosa.segment.agglomerative(rms, 20)

    return segments_timbre, segments_harmony, segments_loudness

def analyze_sound_texture(y_mono, sr):
    # Calculate the short-time Fourier transform (STFT)
    hop_length = 512
    S = np.abs(librosa.stft(y_mono, hop_length=hop_length))
    
    # Find peaks in the STFT
    peaks, _ = find_peaks(S.flatten())
    
    # Estimate polyphony as the number of peaks divided by the number of time frames
    polyphony_estimate = len(peaks) / S.shape[1]
    return polyphony_estimate

def analyze_rhythmic_complexity(y_mono, sr):
    # Calculate the short-time Fourier transform (STFT)
    hop_length = 512
    S = np.abs(librosa.stft(y_mono, hop_length=hop_length))
    
    # Calculate the spectral contrast
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    
    # Calculate the rhythmic entropy and contrast
    rhythmic_entropy = np.mean(-np.sum(contrast * np.log2(contrast + 1e-6), axis=0))
    rhythmic_contrast = np.mean(np.std(contrast, axis=0))
    
    return rhythmic_entropy, rhythmic_contrast

def analyze_chromatic_analysis(y_mono, sr):
    chroma = librosa.feature.chroma_stft(y=y_mono, sr=sr)
    chroma_diff = np.diff(chroma, axis=1)
    chroma_changes = np.mean(np.linalg.norm(chroma_diff, axis=0))
    return chroma_changes

def custom_round(tempo):
    frac, whole = np.modf(tempo)
    if frac < 0.5:
        return whole
    elif 0.5 <= frac < 0.79:
        return whole + 0.5
    else:
        return whole + 1

# Основная функция analyze_audio
def analyze_audio(file_content):
    y, sr = librosa.load(file_content, sr=None, mono=False)

    # Основной анализ
    y_mono = librosa.to_mono(y)
    rms = np.sqrt(np.mean(y_mono**2))
    duration = librosa.get_duration(y=y_mono, sr=sr)
    sample_rate = sr
    meter = Meter(sr)
    loudness = meter.integrated_loudness(y_mono)
    true_peak = np.max(np.abs(y_mono))

    # Стерео анализ
    panning = separation.bss_eval_sources(y, y, compute_permutation=False)[2]
    channel_correlation = np.corrcoef(y[0], y[1])[0][1]

    # Распределение частот и спектральные характеристики
    f, Pxx = scipy.signal.welch(y_mono, sr, nperseg=1024)
    low_energy = np.sum(Pxx[(f >= 20) & (f <= 250)])
    mid_energy = np.sum(Pxx[(f > 250) & (f <= 4000)])
    high_energy = np.sum(Pxx[(f > 4000) & (f <= 20000)])

    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y_mono, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_mono, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y_mono, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y_mono, sr=sr))

    # Транзиенты и мягкость
    o_env = librosa.onset.onset_strength(y=y_mono, sr=sr)
    transients = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr)
    softness = np.mean(librosa.feature.tonnetz(y=y_mono, sr=sr))

    # Тональность и BPM
    chroma = librosa.feature.chroma_stft(y=y_mono, sr=sr)
    key, key_strength = stats.mode(np.argmax(chroma, axis=0), keepdims=False)
    tempo, beat_frames = librosa.beat.beat_track(y=y_mono, sr=sr)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    if 84 <= tempo <= 90:
        tempo = tempo * 2

    # Округление темпа с использованием нашей пользовательской функции
    tempo = custom_round(tempo)

    # Вызов функций анализа
    panning, channel_correlation = analyze_stereo(y)
    loudness = analyze_loudness_ebu(y_mono, sr)
    lra = dynamic_range_analysis(y_mono, sr)
    key, key_strength = harmonic_analysis(y_mono, sr)
    mean_attack_time, mean_release_time = analyze_attack_release(y_mono, sr)
    curvature, flatness = analyze_waveform_shape(y_mono)
    volume_variation = analyze_volume_variation(y_mono)
    segments_timbre, segments_harmony, segments_loudness = analyze_music_structure(y_mono, sr)
    polyphony = analyze_sound_texture(y_mono, sr)
    rhythmic_entropy, rhythmic_contrast = analyze_rhythmic_complexity(y_mono, sr)
    chroma_changes = analyze_chromatic_analysis(y_mono, sr)

    # Вывод результатов
    print(f"Длительность аудио трека: {duration:.2f} секунд")
    print(f"Среднеквадратичное значение амплитуды: {rms:.4f}")
    print(f"Частота дискретизации: {sample_rate} Гц")
    print(f"Панорамирование: {panning:.4f}")
    print(f"Корреляция каналов: {channel_correlation:.4f}")
    print(f"Отклонение громкости от цели (0 LUFS): {loudness:.2f} LU")
    print(f"Динамический диапазон (LRA): {lra:.2f} LU")
    print(f"Тональность: {key}, сила тональности: {key_strength:.4f}")
    print(f"Среднее время атаки: {mean_attack_time:.4f} сек")
    print(f"Среднее время релиза: {mean_release_time:.4f} сек")
    print(f"Кривизна: {curvature:.4f}")
    print(f"Равномерность: {flatness:.4f}")
    print(f"Коэффициент вариации громкости: {volume_variation:.4f}")
    print(f"Сегменты тембра: {segments_timbre}")
    print(f"Сегменты гармонии: {segments_harmony}")
    print(f"Сегменты громкости: {segments_loudness}")
    print(f"Среднее количество пересекающихся голосов (полифония): {polyphony:.4f}")
    print(f"Ритмическая энтропия: {rhythmic_entropy:.4f}")
    print(f"Ритмический контраст: {rhythmic_contrast:.4f}")
    print(f"Изменение хроматических нот: {chroma_changes:.4f}")
    print(f"True Peak: {true_peak:.4f}")
    print(f"Энергия низких частот: {low_energy:.4f}")
    print(f"Энергия средних частот: {mid_energy:.4f}")
    print(f"Энергия высоких частот: {high_energy:.4f}")
    print(f"Спектральный центроид: {spectral_centroid:.2f} Гц")
    print(f"Спектральный спад: {spectral_rolloff:.2f} Гц")
    print(f"Спектральный контраст: {spectral_contrast:.2f} dB")
    print(f"Спектральная ширина: {spectral_bandwidth:.2f} Гц")
    print(f"Мягкость: {softness:.4f}")
    print(f"Тональность: {librosa.core.midi_to_note(key)}")
    print(f"Сила тональности: {key_strength:.2f}")
    print(f"Темп: {tempo:.2f} BPM")

# Загрузка файла и вызов функции analyze_audio
from google.colab import files

uploaded = files.upload()

for name, file_content in uploaded.items():
    print(f"Анализ аудио файла: {name}")
    analyze_audio(io.BytesIO(file_content))
    print("\n")
